use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge valid score

In [6]:
FIELD_NAME = 'sasrec_feat_scores_3'

In [7]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [8]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [9]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_DE/product_embeddings_2023-06-03-23-25-28.pt'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_DE/valid_embeddings_2023-06-03-23-25-35.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_JP/product_embeddings_2023-06-03-23-26-35.pt'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_JP/valid_embeddings_2023-06-03-23-26-42.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_UK/product_embeddings_2023-06-03-23-27-47.pt'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SASRec_Next_Feat/kdd_cup_2023_UK/valid_embeddings_2023-06-03-23-27-54.pt'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [13]:
# sess embeddings 
valid_DE_query_emb = torch.load(DE_valid_embeddings_path, map_location='cpu')
valid_JP_query_emb = torch.load(JP_valid_embeddings_path, map_location='cpu')
valid_UK_query_emb = torch.load(UK_valid_embeddings_path, map_location='cpu')
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [14]:
# product_embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [15]:
merged_candidates_sasrec = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [16]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [17]:
# merged_candidates_sasrec_g = cudf.from_pandas(merged_candidates_sasrec)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
# merged_candidates_sasrec_score_g = merged_candidates_sasrec_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_sasrec_score_g['dataset_id'] = merged_candidates_sasrec_score_g['dataset_id'].fillna(0)
# merged_candidates_sasrec_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_sasrec_score_g = merged_candidates_sasrec_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_sasrec_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_sasrec_score = merged_candidates_sasrec_score_g.to_pandas()
# assert len(merged_candidates_sasrec_score) == len(merged_candidates)

In [19]:
merged_candidates_sasrec_score = merged_candidates_sasrec.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_sasrec_score['dataset_id'] = merged_candidates_sasrec_score['dataset_id'].fillna(0)
merged_candidates_sasrec_score.drop(columns=['locale'], inplace=True)
merged_candidates_sasrec_score = merged_candidates_sasrec_score.sort_values(by=['sess_id', 'product'])
merged_candidates_sasrec_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_sasrec_score) == len(merged_candidates)

In [20]:
# del merged_candidates_sasrec_g
# del product_id_df_g
# del merged_candidates_sasrec_score_g

In [21]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \
        merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_109511/2146751149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \


In [22]:
valid_query_embeddings = valid_query_embeddings.to('cuda:0')
product_embeddings = product_embeddings.to('cuda:0')

In [23]:
merged_candidates_sasrec_score[FIELD_NAME] = get_scores(merged_candidates_sasrec_score, valid_query_embeddings, product_embeddings)

  0%|                                                                                                                                                   | 0/38498 [00:00<?, ?it/s]/tmp/ipykernel_109511/150055658.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38498/38498 [01:17<00:00, 494.25it/s]


In [24]:
normalize_scores(merged_candidates_sasrec_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [25]:
merged_candidates[FIELD_NAME] = merged_candidates_sasrec_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_sasrec_score['normalized_'+FIELD_NAME]

In [26]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [27]:
merged_candidates_sasrec_score

,sess_id,sess_locale,product,dataset_id,sasrec_feat_scores_3,exp_score,normalized_sasrec_feat_scores_3,score_sum
0,0,DE,355165591X,16294.0,7.676408,2156.858004,1.545921e-07,1.395193e+10
1,0,DE,3833237058,32078.0,12.740143,341172.271006,2.445341e-05,1.395193e+10
2,0,DE,B00CIXSI6U,284040.0,5.037027,154.011513,1.103872e-08,1.395193e+10
3,0,DE,B00NVDOWUW,1746.0,12.718519,333874.085318,2.393031e-05,1.395193e+10
4,0,DE,B00NVDP3ZU,106496.0,12.651755,312311.177316,2.238480e-05,1.395193e+10
...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,1162096.0,8.914206,7436.871567,6.403708e-04,1.161338e+07
78842195,261815,UK,B0BCX6QB4L,1026202.0,11.443731,93314.548577,8.035088e-03,1.161338e+07
78842196,261815,UK,B0BFPJYXQL,1055763.0,4.456316,86.169516,7.419846e-06,1.161338e+07
78842197,261815,UK,B0BH3X67S3,966272.0,7.366755,1582.491030,1.362644e-04,1.161338e+07


In [28]:
# # verify sasrec scores
merged_candidates[merged_candidates['sess_id'] == 2000].sort_values(by=['sasrec_scores_3'], ascending=False)[['sess_id', 'sess_locale', 'product', 'sasrec_scores_2', 'normalized_'+FIELD_NAME, FIELD_NAME, 'normalized_sasrec_scores_3']].iloc[:20]

,sess_id,sess_locale,product,sasrec_scores_2,normalized_sasrec_feat_scores_3,sasrec_feat_scores_3,normalized_sasrec_scores_3
596023,2000,DE,B09D3N56DD,16.126688,0.470264,18.453194,0.429727
596028,2000,DE,B09D3P5LYN,14.356667,0.116808,17.060431,0.104548
596064,2000,DE,B09X7BK27V,13.219624,0.032608,15.784467,0.038561
596020,2000,DE,B09D3LP52K,12.823625,0.046279,16.134596,0.035548
595970,2000,DE,B08GYKNCCP,13.141228,0.040386,15.998372,0.035340
596026,2000,DE,B09D3NPHLP,13.011024,0.019759,15.283525,0.026086
596072,2000,DE,B09X7CRKRZ,12.562859,0.019450,15.267721,0.019405
595994,2000,DE,B08TJRVWV1,11.837065,0.012032,14.787446,0.018766
596068,2000,DE,B09X7C7LL1,12.590157,0.010334,14.635305,0.017070
596024,2000,DE,B09D3N8XBH,12.032446,0.016131,15.080639,0.015340


In [29]:
merged_candidates

,sess_id,sess_locale,product,target,sess_avg_price,product_price,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,...,co_graph_counts_2,normalized_co_graph_counts_2,cos_text_bert_scores,text_bert_scores,normalized_text_bert_scores,roberta_scores,normalized_roberta_scores,product_freq,sasrec_feat_scores_3,normalized_sasrec_feat_scores_3
0,0,DE,355165591X,0.0,43.256542,8.990000,2.230508,7.658405e-09,0.512931,1.377575e-09,...,0.0,0.000000,0.903757,378.286041,1.296655e-08,276.525787,7.975509e-07,63.0,7.676408,1.545921e-07
1,0,DE,3833237058,0.0,43.256542,22.000000,9.605231,1.221631e-05,9.325538,9.255110e-06,...,0.0,0.000000,0.921604,387.624756,1.474268e-04,284.460052,2.226209e-03,128.0,12.740143,2.445341e-05
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,0.714114,1.681035e-09,-0.115904,7.345399e-10,...,0.0,0.000000,0.901061,374.802551,3.980740e-10,278.039612,3.624132e-06,7.0,5.037027,1.103872e-08
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,8.750996,5.199363e-06,8.507557,4.084482e-06,...,0.0,0.000000,0.927298,385.701782,2.154962e-05,285.239197,4.852260e-03,197.0,12.718519,2.393031e-05
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,8.056712,2.596729e-06,5.898870,3.007453e-07,...,0.0,0.000000,0.930655,385.398499,1.591202e-05,284.763611,3.015780e-03,639.0,12.651755,2.238480e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,6.813615,1.076201e-03,7.203015,4.597607e-04,...,0.0,0.000000,0.972680,438.956238,1.636532e-04,281.902344,1.476179e-04,7.0,8.914206,6.403708e-04
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,9.030836,9.881445e-03,10.123234,8.526421e-03,...,2.0,0.009091,0.972680,438.956238,1.636532e-04,281.902344,1.476179e-04,63.0,11.443731,8.035088e-03
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,0.796892,2.623396e-06,1.711608,1.895152e-06,...,0.0,0.000000,0.953467,430.164368,2.486932e-08,283.306732,6.012531e-04,7.0,4.456316,7.419846e-06
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,4.250781,8.296004e-05,6.447586,2.159998e-04,...,0.0,0.000000,0.961829,434.029083,1.186011e-06,273.954742,5.218429e-08,61.0,7.366755,1.362644e-04
